<a href="https://www.kaggle.com/code/karma12/multilabel-classification-with-huggingface-bert?scriptVersionId=140864545" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q datasets
!pip install -q transformers
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 6.4 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3


In [3]:
from datasets import load_dataset,DatasetDict
from transformers import (AutoTokenizer,AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)
import torch

dataset = load_dataset("csv", data_files="/kaggle/input/multilabel-classification-dataset/train.csv")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c94e8593bfd251f3/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
ds_train_devtest = dataset['train'].train_test_split(test_size=0.2, seed=42)
ds_devtest = ds_train_devtest['test'].train_test_split(test_size=0.5, seed=42)


dataset = DatasetDict({
    'train': ds_train_devtest['train'],
    'valid': ds_devtest['train'],
    'test': ds_devtest['test']
})

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance'],
        num_rows: 16777
    })
    valid: Dataset({
        features: ['ID', 'TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance'],
        num_rows: 2097
    })
    test: Dataset({
        features: ['ID', 'TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance'],
        num_rows: 2098
    })
})

In [6]:
dataset['train'][0]

{'ID': 11567,
 'TITLE': 'Classification of rank two Lie conformal algebras',
 'ABSTRACT': '  We give a complete classification (up to isomorphism) of Lie conformal\nalgebras which are free of rank two as $\\C[\\partial]$-modules, and determine\ntheir automorphism groups.\n',
 'Computer Science': 0,
 'Physics': 0,
 'Mathematics': 1,
 'Statistics': 0,
 'Quantitative Biology': 0,
 'Quantitative Finance': 0}

In [7]:
dataset.shape

{'train': (16777, 9), 'valid': (2097, 9), 'test': (2098, 9)}

In [8]:
dataset.column_names

{'train': ['ID',
  'TITLE',
  'ABSTRACT',
  'Computer Science',
  'Physics',
  'Mathematics',
  'Statistics',
  'Quantitative Biology',
  'Quantitative Finance'],
 'valid': ['ID',
  'TITLE',
  'ABSTRACT',
  'Computer Science',
  'Physics',
  'Mathematics',
  'Statistics',
  'Quantitative Biology',
  'Quantitative Finance'],
 'test': ['ID',
  'TITLE',
  'ABSTRACT',
  'Computer Science',
  'Physics',
  'Mathematics',
  'Statistics',
  'Quantitative Biology',
  'Quantitative Finance']}

In [9]:
dataset = dataset.remove_columns(['ID','TITLE'])

In [10]:
dataset['train'][0]

{'ABSTRACT': '  We give a complete classification (up to isomorphism) of Lie conformal\nalgebras which are free of rank two as $\\C[\\partial]$-modules, and determine\ntheir automorphism groups.\n',
 'Computer Science': 0,
 'Physics': 0,
 'Mathematics': 1,
 'Statistics': 0,
 'Quantitative Biology': 0,
 'Quantitative Finance': 0}

In [11]:
# create labels column
cols = dataset["train"].column_names
dataset = dataset.map(lambda x : {"labels": [x[c] for c in cols if c != "ABSTRACT"]})
dataset

  0%|          | 0/16777 [00:00<?, ?ex/s]

  0%|          | 0/2097 [00:00<?, ?ex/s]

  0%|          | 0/2098 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['ABSTRACT', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance', 'labels'],
        num_rows: 16777
    })
    valid: Dataset({
        features: ['ABSTRACT', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance', 'labels'],
        num_rows: 2097
    })
    test: Dataset({
        features: ['ABSTRACT', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance', 'labels'],
        num_rows: 2098
    })
})

In [12]:
dataset['train'][2]

{'ABSTRACT': '  We describe a communication game, and a conjecture about this game, whose\nproof would imply the well-known Sensitivity Conjecture asserting a polynomial\nrelation between sensitivity and block sensitivity for Boolean functions. The\nauthor defined this game and observed the connection in Dec. 2013 - Jan. 2014.\nThe game and connection were independently discovered by Gilmer, Koucký, and\nSaks, who also established further results about the game (not proved by us)\nand published their results in ITCS \'15 [GKS15].\nThis note records our independent work, including some observations that did\nnot appear in [GKS15]. Namely, the main conjecture about this communication\ngame would imply not only the Sensitivity Conjecture, but also a stronger\nhypothesis raised by Chung, Füredi, Graham, and Seymour [CFGS88]; and,\nanother related conjecture we pose about a "query-bounded" variant of our\ncommunication game would suffice to answer a question of Aaronson, Ambainis,\nBalodis,

In [13]:
model_ckpt = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, problem_type="multi-label-classification")

In [14]:
def tokenize_and_encode(examples):
    return tokenizer(examples["ABSTRACT"], truncation=True)

In [15]:
cols = dataset["train"].column_names
cols.remove("labels")
ds_enc = dataset.map(tokenize_and_encode, batched=True, remove_columns=cols)
ds_enc

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16777
    })
    valid: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2097
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2098
    })
})

In [16]:
ds_enc['train']['labels'][0]

[0, 0, 1, 0, 0, 0]

In [17]:
# cast label IDs to floats
ds_enc.set_format("torch")
ds_enc = (ds_enc
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

  0%|          | 0/16777 [00:00<?, ?ex/s]

  0%|          | 0/2097 [00:00<?, ?ex/s]

  0%|          | 0/2098 [00:00<?, ?ex/s]

In [18]:
num_labels = 6
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=num_labels, problem_type='multi_label_classification').to('cuda')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [19]:
ds_enc["train"][0]

{'input_ids': tensor([  101,  2057,  2507,  1037,  3143,  5579,  1006,  2039,  2000, 11163,
         19539,  1007,  1997,  4682, 23758,  2389, 11208,  2015,  2029,  2024,
          2489,  1997,  4635,  2048,  2004,  1002,  1032,  1039,  1031,  1032,
          7704,  1033,  1002,  1011, 14184,  1010,  1998,  5646,  2037,  8285,
         19539,  2967,  1012,   102]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor([0., 0., 1., 0., 0., 0.])}

In [20]:
args = TrainingArguments(
    '/kaggle/working/',
    evaluation_strategy = "epoch",
    num_train_epochs=5,
)

In [21]:
trainer = Trainer(model=model, args = args,
                  train_dataset = ds_enc["train"],
                  eval_dataset = ds_enc["test"],
                  tokenizer=tokenizer)

In [22]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.189400,0.184107
2,0.144200,0.167039
3,0.107800,0.190839
4,0.070300,0.220338
5,0.042200,0.240762


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=5245, training_loss=0.1156929901603066, metrics={'train_runtime': 3877.973, 'train_samples_per_second': 21.631, 'train_steps_per_second': 1.353, 'total_flos': 1.6816351628021148e+16, 'train_loss': 0.1156929901603066, 'epoch': 5.0})

In [23]:
trainer.evaluate()

{'eval_loss': 0.24076198041439056,
 'eval_runtime': 31.0462,
 'eval_samples_per_second': 67.577,
 'eval_steps_per_second': 4.252,
 'epoch': 5.0}

In [46]:
dataset['train'][700]

{'ABSTRACT': '  In this paper, we introduce the concept of a virtual machine with\ngraph-organised memory as a versatile backend for both explicit-state and\nabstraction-driven verification of software. Our virtual machine uses the LLVM\nIR as its instruction set, enriched with a small set of hypercalls. We show\nthat the provided hypercalls are sufficient to implement a small operating\nsystem, which can then be linked with applications to provide a\nPOSIX-compatible verification environment. Finally, we demonstrate the\nviability of the approach through a comparison with a more\ntraditionally-designed LLVM model checker.\n',
 'Computer Science': 1,
 'Physics': 0,
 'Mathematics': 0,
 'Statistics': 0,
 'Quantitative Biology': 0,
 'Quantitative Finance': 0,
 'labels': [1, 0, 0, 0, 0, 0]}

In [47]:
# text = '  In some planetary systems the orbital periods of two of its members present a\ncommensurability, usually known by mean-motion resonance. These resonances\ngreatly enhance the mutual gravitational influence of the planets. As a\nconsequence, these systems present uncommon behaviours and their motions need\nto be studied with specific methods. Some features are unique and allow us a\nbetter understanding and characterisation of these systems. Moreover,\nmean-motion resonances are a result of an early migration of the orbits in an\naccretion disk, so it is possible to derive constraints on their formation.\nHere we review the dynamics of a pair of resonant planets and explain how their\norbits evolve in time. We apply our results to the HD45365 planetary system\n'
text = '  In this paper, we introduce the concept of a virtual machine with\ngraph-organised memory as a versatile backend for both explicit-state and\nabstraction-driven verification of software. Our virtual machine uses the LLVM\nIR as its instruction set, enriched with a small set of hypercalls. We show\nthat the provided hypercalls are sufficient to implement a small operating\nsystem, which can then be linked with applications to provide a\nPOSIX-compatible verification environment. Finally, we demonstrate the\nviability of the approach through a comparison with a more\ntraditionally-designed LLVM model checker.\n'
encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [48]:
logits = outputs.logits
logits.shape

torch.Size([1, 6])

In [60]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
predictions
# turn predicted id's into actual label names
ans_cols = cols[1:]
predicted_labels = [ans_cols[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['Computer Science']


In [61]:
# model 2

In [62]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    print("sigmoid",sigmoid)
    probs = sigmoid(torch.Tensor(predictions))
    print("probs",probs)
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    print("y_pred",y_pred)
    y_pred[np.where(probs >= threshold)] = 1
    print("y_pred_1",y_pred)
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [68]:
ds_enc['train']['input_ids'][0]

tensor([  101,  2057,  2507,  1037,  3143,  5579,  1006,  2039,  2000, 11163,
        19539,  1007,  1997,  4682, 23758,  2389, 11208,  2015,  2029,  2024,
         2489,  1997,  4635,  2048,  2004,  1002,  1032,  1039,  1031,  1032,
         7704,  1033,  1002,  1011, 14184,  1010,  1998,  5646,  2037,  8285,
        19539,  2967,  1012,   102])

In [69]:
ds_enc['train']['input_ids'][0].unsqueeze(0)

tensor([[  101,  2057,  2507,  1037,  3143,  5579,  1006,  2039,  2000, 11163,
         19539,  1007,  1997,  4682, 23758,  2389, 11208,  2015,  2029,  2024,
          2489,  1997,  4635,  2048,  2004,  1002,  1032,  1039,  1031,  1032,
          7704,  1033,  1002,  1011, 14184,  1010,  1998,  5646,  2037,  8285,
         19539,  2967,  1012,   102]])

In [70]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [73]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [74]:
#forward pass
outputs = model(input_ids=ds_enc['train']['input_ids'][0].unsqueeze(0).to('cuda'),
                labels=ds_enc['train'][0]['labels'].unsqueeze(0).to('cuda'))
outputs

SequenceClassifierOutput(loss=tensor(0.0028, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-5.6027, -5.7539,  5.6803, -5.5725, -6.3632, -6.7545]],
       device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [76]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [78]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.078200,0.250742,0.808495,0.883786,0.638704
2,0.057100,0.272240,0.827096,0.896885,0.677312
3,0.031500,0.320527,0.815628,0.890983,0.647760
4,0.015500,0.359414,0.819290,0.895167,0.656816
5,0.006900,0.375800,0.824249,0.897519,0.663489


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


sigmoid Sigmoid()
probs tensor([[9.8808e-01, 5.6730e-01, 6.6209e-03, 2.9143e-02, 2.0635e-02, 7.9109e-04],
        [5.6997e-01, 1.8960e-01, 1.2239e-01, 7.3050e-01, 1.9903e-01, 4.0617e-04],
        [1.0781e-02, 3.1443e-03, 9.9793e-01, 2.7454e-03, 3.5462e-04, 3.8200e-04],
        ...,
        [8.0721e-01, 4.9409e-04, 1.1062e-02, 9.5897e-01, 2.3830e-04, 9.6171e-05],
        [5.4293e-01, 8.8574e-04, 1.5763e-01, 9.9137e-01, 2.5013e-04, 1.3697e-04],
        [3.0314e-02, 1.3234e-02, 8.3008e-02, 9.9719e-01, 3.1720e-03, 2.0358e-04]])
y_pred [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
y_pred_1 [[1. 1. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


sigmoid Sigmoid()
probs tensor([[7.8551e-01, 5.7462e-02, 1.7944e-03, 3.4788e-02, 6.0379e-01, 8.0703e-04],
        [3.7025e-01, 2.7682e-01, 2.3877e-02, 1.6597e-01, 8.4923e-01, 8.2729e-04],
        [2.3402e-02, 2.2716e-03, 9.9757e-01, 2.9035e-04, 3.2008e-04, 2.5953e-04],
        ...,
        [9.9640e-01, 3.2075e-04, 1.2048e-03, 7.5818e-01, 9.8154e-04, 2.4931e-04],
        [9.5247e-01, 1.3292e-04, 7.5240e-01, 9.8137e-01, 3.5033e-04, 2.4312e-04],
        [9.1088e-03, 4.5351e-01, 1.8026e-03, 9.9689e-01, 6.8773e-03, 1.9158e-03]])
y_pred [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
y_pred_1 [[1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


...

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


sigmoid Sigmoid()
probs tensor([[9.9099e-01, 8.8203e-01, 2.0503e-03, 3.3800e-04, 1.1208e-01, 9.5008e-04],
        [2.2593e-01, 8.7350e-01, 2.3185e-02, 8.1691e-03, 8.8863e-01, 3.0959e-04],
        [9.2765e-04, 7.2106e-04, 9.9889e-01, 6.3755e-04, 2.0853e-04, 1.6989e-04],
        ...,
        [9.9885e-01, 2.1889e-04, 1.2029e-03, 9.9759e-01, 7.4131e-04, 8.2761e-04],
        [9.8902e-01, 2.3917e-04, 6.8407e-01, 9.9940e-01, 5.3073e-04, 9.3450e-04],
        [1.2489e-03, 5.9894e-03, 9.0827e-04, 9.9858e-01, 6.6498e-03, 1.5016e-03]])
y_pred [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
y_pred_1 [[1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


sigmoid Sigmoid()
probs tensor([[9.9950e-01, 2.5677e-01, 9.5177e-04, 2.8084e-04, 9.7818e-04, 4.1066e-04],
        [9.5111e-01, 8.9328e-01, 5.9711e-02, 5.4450e-01, 4.0993e-01, 2.2695e-04],
        [1.3250e-03, 3.7777e-04, 9.9905e-01, 4.6506e-04, 2.3130e-04, 1.3939e-04],
        ...,
        [9.9951e-01, 2.1965e-04, 1.1716e-03, 9.9734e-01, 6.7416e-04, 6.1778e-04],
        [9.9932e-01, 3.6875e-04, 4.8775e-01, 9.9918e-01, 1.1344e-03, 1.3887e-03],
        [1.5513e-03, 1.6196e-03, 1.9343e-03, 9.9912e-01, 1.3185e-03, 7.3640e-04]])
y_pred [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
y_pred_1 [[1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


TrainOutput(global_step=5245, training_loss=0.03659175537789402, metrics={'train_runtime': 3802.4659, 'train_samples_per_second': 22.061, 'train_steps_per_second': 1.379, 'total_flos': 1.6832817215862588e+16, 'train_loss': 0.03659175537789402, 'epoch': 5.0})

In [79]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


sigmoid Sigmoid()
probs tensor([[9.9950e-01, 2.5677e-01, 9.5177e-04, 2.8084e-04, 9.7818e-04, 4.1066e-04],
        [9.5111e-01, 8.9328e-01, 5.9711e-02, 5.4450e-01, 4.0993e-01, 2.2695e-04],
        [1.3250e-03, 3.7777e-04, 9.9905e-01, 4.6506e-04, 2.3130e-04, 1.3939e-04],
        ...,
        [9.9951e-01, 2.1965e-04, 1.1716e-03, 9.9734e-01, 6.7416e-04, 6.1778e-04],
        [9.9932e-01, 3.6875e-04, 4.8775e-01, 9.9918e-01, 1.1344e-03, 1.3887e-03],
        [1.5513e-03, 1.6196e-03, 1.9343e-03, 9.9912e-01, 1.3185e-03, 7.3640e-04]])
y_pred [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
y_pred_1 [[1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


{'eval_loss': 0.37580034136772156,
 'eval_f1': 0.8242491657397109,
 'eval_roc_auc': 0.8975187463695411,
 'eval_accuracy': 0.663489037178265,
 'eval_runtime': 33.2051,
 'eval_samples_per_second': 63.183,
 'eval_steps_per_second': 3.975,
 'epoch': 5.0}

In [80]:
text = '  In this paper, we introduce the concept of a virtual machine with\ngraph-organised memory as a versatile backend for both explicit-state and\nabstraction-driven verification of software. Our virtual machine uses the LLVM\nIR as its instruction set, enriched with a small set of hypercalls. We show\nthat the provided hypercalls are sufficient to implement a small operating\nsystem, which can then be linked with applications to provide a\nPOSIX-compatible verification environment. Finally, we demonstrate the\nviability of the approach through a comparison with a more\ntraditionally-designed LLVM model checker.\n'
encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)
logits = outputs.logits
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
predictions
# turn predicted id's into actual label names
ans_cols = cols[1:]
predicted_labels = [ans_cols[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['Computer Science']
